In [ ]:
from google.colab import files
import pandas as pd
uploaded = files.upload()

Saving tweets.csv to tweets.csv


In [ ]:
import pandas as pd
import sqlite3
from textblob import TextBlob
import re

# Step 1: Load your CSV
df = pd.read_csv("tweets.csv")

# Step 2: Define a cleaning function
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)   # remove URLs
    text = re.sub(r'@\w+|\#','', text)  # remove mentions & hashtags
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # remove special chars
    return text.strip()

# Step 3: Apply cleaning → new column
df["Description_Clean"] = df["text"].apply(clean_text)

# Step 4: Sentiment analysis
def analyze_sentiment(text):
    analysis = TextBlob(str(text))
    polarity = analysis.sentiment.polarity
    if polarity > 0:
        sentiment = "Positive"
    elif polarity < 0:
        sentiment = "Negative"
    else:
        sentiment = "Neutral"
    return polarity, sentiment

df["Polarity"], df["Sentiment"] = zip(*df["Description_Clean"].apply(analyze_sentiment))

# Step 5: Save everything into SQLite DB
conn = sqlite3.connect("modified_tweets.db")
df.to_sql("tweets_sentiment", conn, if_exists="replace", index=False)
conn.close()

print("✅ All columns (original, cleaned, polarity, sentiment) stored in 'tweets_sentiment' table inside modified_tweets.db")


✅ All columns (original, cleaned, polarity, sentiment) stored in 'tweets_sentiment' table inside modified_tweets.db


In [ ]:
df.head(10)

,id,keyword,location,text,target,Description_Clean,Polarity,Sentiment
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1,communal violence in bhainsa telangana stones ...,0.000000,Neutral
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1,telangana section 144 has been imposed in bhai...,0.000000,Neutral
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1,arsonist sets cars ablaze at dealership,0.000000,Neutral
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1,arsonist sets cars ablaze at dealership,0.000000,Neutral
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0,lord jesus your love brings freedom and pardon...,0.500000,Positive
5,5,ablaze,OC,"If this child was Chinese, this tweet would ha...",0,if this child was chinese this tweet would hav...,0.016667,Positive
6,6,ablaze,"London, England",Several houses have been set ablaze in Ngemsib...,1,several houses have been set ablaze in ngemsib...,0.000000,Neutral
7,7,ablaze,Bharat,Asansol: A BJP office in Salanpur village was ...,1,asansol a bjp office in salanpur village was s...,-0.166667,Negative
8,8,ablaze,"Accra, Ghana","National Security Minister, Kan Dapaah's side ...",0,national security minister kan dapaahs side ch...,0.400000,Positive
9,9,ablaze,Searching,This creature who’s soul is no longer clarent ...,0,this creature whos soul is no longer clarent b...,0.000000,Neutral


In [ ]:
import pandas as pd
import sqlite3
from textblob import TextBlob
import re
from concurrent.futures import ProcessPoolExecutor

# Step 1: Load your CSV
df = pd.read_csv("tweets.csv")

# Step 2: Define cleaning function
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)   # remove URLs
    text = re.sub(r'@\w+|\#','', text)  # remove mentions & hashtags
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # remove special chars
    return text.strip()

df["Description_Clean"] = df["text"].apply(clean_text)

# Step 3: Sentiment analysis function
def analyze_sentiment(text):
    analysis = TextBlob(str(text))
    polarity = analysis.sentiment.polarity
    if polarity > 0:
        sentiment = "Positive"
    elif polarity < 0:
        sentiment = "Negative"
    else:
        sentiment = "Neutral"
    return polarity, sentiment

# Step 4: Apply parallel sentiment analysis
def parallel_sentiment(texts):
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(analyze_sentiment, texts))
    return results

results = parallel_sentiment(df["Description_Clean"].tolist())

# Step 5: Assign back results
df["Polarity"], df["Sentiment"] = zip(*results)

# Step 6: Save into SQLite database
conn = sqlite3.connect("modified_tweets.db")
df.to_sql("tweets_sentiment", conn, if_exists="replace", index=False)
conn.close()

print("Parallel sentiment analysis completed and saved in DB")


✅ Parallel sentiment analysis completed and saved in DB


In [ ]:
df.head(10)

,id,keyword,location,text,target,Description_Clean,Polarity,Sentiment
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1,communal violence in bhainsa telangana stones ...,0.000000,Neutral
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1,telangana section 144 has been imposed in bhai...,0.000000,Neutral
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1,arsonist sets cars ablaze at dealership,0.000000,Neutral
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1,arsonist sets cars ablaze at dealership,0.000000,Neutral
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0,lord jesus your love brings freedom and pardon...,0.500000,Positive
5,5,ablaze,OC,"If this child was Chinese, this tweet would ha...",0,if this child was chinese this tweet would hav...,0.016667,Positive
6,6,ablaze,"London, England",Several houses have been set ablaze in Ngemsib...,1,several houses have been set ablaze in ngemsib...,0.000000,Neutral
7,7,ablaze,Bharat,Asansol: A BJP office in Salanpur village was ...,1,asansol a bjp office in salanpur village was s...,-0.166667,Negative
8,8,ablaze,"Accra, Ghana","National Security Minister, Kan Dapaah's side ...",0,national security minister kan dapaahs side ch...,0.400000,Positive
9,9,ablaze,Searching,This creature who’s soul is no longer clarent ...,0,this creature whos soul is no longer clarent b...,0.000000,Neutral


In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email_alert(to_email, subject, message):
    # Gmail SMTP settings
    smtp_server = "smtp.gmail.com"
    smtp_port = 587

    # Your Gmail + App Password
    sender_email = "keerthanareddy2006@gmail.com"
    sender_password = "lwkkxdgrwvmfgski"  # App password

    # Create email
    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = to_email
    msg["Subject"] = subject
    msg.attach(MIMEText(message, "plain"))

    try:
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, to_email, msg.as_string())
        print(f"✅ Email sent to {to_email}")
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        try:
            server.quit()
        except:
            pass


# 🔹 Generate sentiment summary from your DataFrame
def create_sentiment_summary(df):
    sentiment_counts = df["Sentiment"].value_counts().to_dict()
    total = len(df)

    summary_text = f"""
Sentiment Analysis Report 📊

Total Tweets Analyzed: {total}

Positive Tweets: {sentiment_counts.get('Positive', 0)}
Negative Tweets: {sentiment_counts.get('Negative', 0)}
Neutral Tweets: {sentiment_counts.get('Neutral', 0)}

✅ Detailed results are stored in the database (modified_tweets.db, table: tweets_sentiment).
"""
    return summary_text


# 🔹 Example usage after your analysis is done
if __name__ == "__main__":
    import pandas as pd
    import sqlite3

    # Load your sentiment results from DB
    conn = sqlite3.connect("modified_tweets.db")
    df = pd.read_sql("SELECT * FROM tweets_sentiment", conn)
    conn.close()

    # Create summary
    report = create_sentiment_summary(df)

    # Send mail
    send_email_alert(
        to_email="keerthanareddy2006@gmail.com",
        subject="📊 Sentiment Analysis Summary",
        message=report
    )


✅ Email sent to keerthanareddy2006@gmail.com
